## Auteur : Mauchaussee Pauline 
# Date de création : 2023/08/17  
# Présentation :** Ce notebook permet de telecharger des données climatique depuis une liste de stations stockée dans un table BigQuerry et d'envoyer les données en format CSV dans un bucket GCP.

## Prérequis :
# - Sauvegarder le fichier excel sous forme de CSV
# - Un bucket gcp pour le stockage de données. (BUCKET_NAME)
# - Une table BigQuerry contenant la liste des station (TABLE_ID)
# - Un token API infoclimat


## Params:
# BUCKET_NAME : nom du bucket GCP, DOIT être dans le même projet.
# TABLE_IDE : ID de la table BigQuerry, DOIT être dans le même projet.
# KEY_PATH : Chemin vers le fichier JSON permettant de ce connecter au service account

## Import des librairies

In [51]:
from google.cloud import storage

In [52]:
import os

In [53]:
#Importer les bibliothèques nécessaires (pandas et pyspark) dans le script pyspark
import pandas as pd

In [54]:
#!pip install openpyxl

In [55]:
#!pip install xlrd

In [56]:
import openpyxl

In [57]:
from pyspark.sql.functions import lit

In [58]:
import pyspark.sql
from pyspark.sql import *

In [59]:
import requests

In [60]:
from pyspark import SparkFiles

In [61]:
import zipfile
import io
import os

# Création d'une fonction pour récupérer de la donnée à partir d'une URL 

In [63]:
def TelechargementData():

    url = "https://services.eaufrance.fr/documents/openData/SISPEA_FR_"+str(annee)+"_AEP.zip"

    

    reponse = requests.get(url)

    content = reponse.content

    # Création d'un objet ZipFile à partir du contenu ZIP téléchargé
    zip_file = zipfile.ZipFile(io.BytesIO(content))

    # Création du répertoire de destination s'il n'existe pas
    os.makedirs("Data", exist_ok=True)

    # Extraction des fichiers du fichier ZIP et enregistrement dans le répertoire de destination
    zip_file.extractall("Data")
    
    # Fermeture du fichier ZIP
    zip_file.close()

# Création d'une fonction pour lire mon fichier excel et conversion en un DataFrame Pandas

In [64]:
def lecturePandas(annee):
    ''' Cette fonction permet de lire un fichier excel et de le convertir en dataFrame Pandas'''

    #Pour lire le fichier excel on utilise Pandas 
    #On donne le chemin du fichier excel
    excel_file_path = "./Data/SISPEA_FR_"+str(annee)+"_AEP.xls"
    #On donne le nom de l'onglet
    sheet_name = "Entités de gestion"
    #Lecture du fichier excel avec pandas 
    pandas_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    return pandas_df

# Je sélectionne les colonnes que l'on souhaite garder

In [75]:
def selectioncolonne(pandas_df):
    '''Cette fonction selectionne les colonnes que l'on veut garder '''
    pandas_df_new = pandas_df[["N° SIREN","VP.224","VP.225", "VP.226", "VP.227","VP.228","VP.229","VP.231", "VP.232","VP.234"]]

    return pandas_df_new

# Création d'une fonction pour convertir la table pandas en table spark

In [76]:
def convertirPandasEnSpark(pandas_df):
    '''Cette fonction converti la table pandas en table spark'''
    #Conversion du DataFrame Pandas en DataFrame Spark
    table = spark.createDataFrame(pandas_df)
    
    return table

# Je renomme les colonnes de mon tableau

In [67]:
def renommageColonnes(table_spark):
    ''' Cette fonction renomme les colonnes de la table '''

    table_renamed = table_spark.withColumnRenamed("VP.224", "Indice linéaire de consommation")\
        .withColumnRenamed("VP.225", "Rendement sur les 3 années précédentes")\
        .withColumnRenamed("VP.226", "Rendement seuil par défaut")\
        .withColumnRenamed("VP.227", "Rendement seuil en ZRE")\
        .withColumnRenamed("VP.228", "Densité linéaire d'abonnés")\
        .withColumnRenamed("VP.229", "Ratio habitants par abonnés")\
        .withColumnRenamed("VP.231", "Consommation moyenne par abonné")\
        .withColumnRenamed("VP.232", "Volumes consommés comptabilisés")\
        .withColumnRenamed("VP.234", "Volume produit + Volume importé")
    return table_renamed

# Ajout de la colonne année

In [68]:
def ajoutcol_annee(table_spark,annee):
    # Add new constanst column
    dataframe_colannee = table_spark.withColumn("Annee", lit(annee))
    return dataframe_colannee

# Je sauvegarde mon tableau dans un fichier csv

In [69]:
def transformationCSV(table_spark):
    ''' Cette fonction enregistre une table spark au format CSV '''
    table_spark.write.csv("./Local Disk/Data/consommation_eau.csv", header=True, mode="append")

# Fonction principale pour lancer toutes mes fonctions

In [78]:
def main(annee):
    
    TelechargementData()
    
    pandas_dataframe = lecturePandas(annee)

    pandas_dataframe_propre = selectioncolonne(pandas_dataframe)

    spark_dataframe = convertirPandasEnSpark(pandas_dataframe_propre)

    df_bons_noms = renommageColonnes(spark_dataframe)
    
    dataframe_colannee = ajoutcol_annee(df_bons_noms, annee)

    transformationCSV(dataframe_colannee)
    

# Selection des années à traiter.

In [79]:
annee_min = 2008
anne_max = 2021

# Execution du programme

In [80]:
for annee in range(annee_min,anne_max+1):
    main(annee)

# Envoie des fichiers csv dans le bucket

In [ ]:
## lien URI du bucket crée 
DATALAKE_PATH = "gs://code_de_source_lake"


In [ ]:
## variables pour envoyer mon fichier csv sur mon bucket

BUCKET_NAME = "code_de_source_lake"
local_folder_path = "./Data/Output_data"
gcs_file_path = "consommation_eau.csv"



In [ ]:
# Nom du dossier dans le bucket GCS
gcs_folder_name = 'Output_data_consommationeau'

In [ ]:
# Initialiser le client de stockage GCP
client = storage.Client()

In [ ]:
# Récupérer le bucket
bucket = client.bucket(BUCKET_NAME)


In [ ]:
# Envoyer le fichier dans le bucket
blob = bucket.blob(gcs_file_path)

In [ ]:
# Parcourir les fichiers du dossier local
for root, dirs, files in os.walk(local_folder_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        gcs_file_path = os.path.join(gcs_folder_name, os.path.relpath(local_file_path, local_folder_path))
        
        # Créer un objet Blob dans le bucket
        blob = bucket.blob(gcs_file_path)
        
        # Charger le contenu du fichier local dans le Blob
        blob.upload_from_filename(local_file_path)

        print(f'Fichier {local_file_path} envoyé vers {gcs_file_path}.')

In [ ]:
# J'ouvre mon fichier pour le lire en binaire
#with open (local_file_path, "rb") as fichier:
#J'upload le contenu de mon fichier dans mon blob
    #blob.upload_from_file(fichier)

#print(f"Le fichier {local_file_path} a été upload dans le bucket {BUCKET_NAME}.")